# 1 图像部分
- pysekiro
    - img_tools
        - \__init__.py
            - adjustment.py (游戏窗口校准)
            - get_status.py (状态获取)
            - get_vertices.py (顶点位置获取)
            - grab_screen.py (屏幕图像抓取)

## 1.1 屏幕图像抓取

查看代码 [grab_screen.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/img_tools/grab_screen.py)

**注：这部分是我从别人那里借鉴而来的**  
[源代码](https://github.com/Sentdex/pygta5/blob/master/grabscreen.py)

我修改了 grab_screen 的返回值
~~~python
return cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
~~~

追加了以下内容
~~~python
# 游戏窗口大小
GAME_WIDTH = 800
GAME_HEIGHT = 450

def get_screen():
    return grab_screen(region=(0, 31, GAME_WIDTH, GAME_HEIGHT+31))    # 游戏最上面31像素是白色边框
~~~

In [ ]:
# 运行以下代码，就会抓取当前的屏幕图像
import cv2
from pysekiro.img_tools.grab_screen import get_screen

screen = get_screen()
cv2.imshow('screen', screen)
cv2.waitKey(0)

# print('图像', screen)
print('形状', screen.shape)

## 1.2 顶点位置获取

查看代码 [get_vertices.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/img_tools/get_vertices.py)

其实就是我无法写出一个自动把血条、架势条抠出来的代码，所以才写的这个程序。  
这个程序的主要作用是，在图像上点击然后返回相关的位置信息。这样就能获取到血条或者架势条在图像中的位置信息。  
以下是把人物抠出来的演示  
![demo.gif](https://github.com/ricagj/pysekiro/blob/main/imgs/demo.gif?raw=true)

如上面的演示所示，在弹出来的窗口中用鼠标左键按顺序依次点击**左下**，**左上**，**右上**，**右下**，一共 4 次，然后按键盘上的“ESC”键，就会自动返回 x, x_w, y, y_h。  
（注意：这个点击的顺序是规定好的，点击的次数也是规定好的）

其中    
x 代表 x轴的起始位置，也是图像的最左边  
y 代表 y轴的起始位置，也是图像的最上边  
x_w 代表 x轴的终止位置，也是图像的最右边  
y_h 代表 y轴的终止位置，也是图像的最下边  

**这些位置信息看起来有点奇怪，是因为图像的原点其实在左上角，想一下矩阵第一行或第一列在什么位置应该就理解了**

In [ ]:
# 参照着上面的演示自己试一下
import cv2

from pysekiro.img_tools.get_vertices import get_xywh

# 抓取当前屏幕显示的图像来试验
from pysekiro.img_tools.grab_screen import get_screen
img = get_screen()

get_xywh(img)

## 1.3 状态获取

查看代码 [get_status.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/img_tools/get_status.py)

![get_status](https://github.com/ricagj/pysekiro/blob/main/imgs/status.jpg?raw=true)

要获取的状态有：自身生命，自身架势，目标生命，目标架势。  
- 获取方法
    1. 靠上面 **1.2顶点位置获取** 把状态所在图像的位置抠出来（得到x, x_w, y, y_h）。
    2. 选取ROI区域（例如生命值所在的图像），进行颜色通道分离，取某颜色通道的图像，然后对图像进行处理。
    3. 量化

### 1.3.1 生命值的获取

1. 截取图像中生命值的区域
2. ~~由于剩余生命值部分呈红色，所以进行颜色通道分离，取R(Red)通道的图像~~
2. 打脸，竟然是G(Green)通道的图像能准确获取数值
3. 通过观察发现，剩余生命值部分的像素点的值集中在一个范围，所以用图像阈值处理，将像素值不在范围内的点的值的设置为0
4. 处理完毕，获取数值

~~~python
# 获取自身生命
def get_Self_HP(img):
    img_roi = roi(img, x=48, x_w=305, y=409, y_h=409+1)    # x, x_w, y, y_h 获取自 get_vertices.py

    b, g ,r =cv2.split(img_roi)    # 颜色通道分离

    retval, img_th = cv2.threshold(g, 50, 255, cv2.THRESH_TOZERO)              # 图像阈值处理，像素点的值低于50的设置为0
    retval, img_th = cv2.threshold(img_th, 70, 255, cv2.THRESH_TOZERO_INV)    # 图像阈值处理，像素点的值高于70的设置为0

    target_img = img_th[0]
    if 0 in target_img:
        Self_HP = np.argmin(target_img)
    else:
        Self_HP = len(target_img)

    return Self_HP

# 获取目标生命
def get_Target_HP(img):
    img_roi = roi(img, x=48, x_w=216, y=41, y_h=41+1)    # x, x_w, y, y_h 获取自 get_vertices.py

    b, g ,r =cv2.split(img_roi)    # 颜色通道分离

    retval, img_th = cv2.threshold(g, 25, 255, cv2.THRESH_TOZERO)             # 图像阈值处理，像素点的值低于25的设置为0
    retval, img_th = cv2.threshold(img_th, 70, 255, cv2.THRESH_TOZERO_INV)    # 图像阈值处理，像素点的值高于70的设置为0

    target_img = img_th[0]
    if 0 in target_img:
        Target_HP = np.argmin(target_img)
    else:
        Target_HP = len(target_img)
    
    return Target_HP
~~~

![demo_get_Self_HP.gif](https://github.com/ricagj/pysekiro/blob/main/imgs/demo_get_Self_HP.gif?raw=true)

- 获取数值
    - 我们所看到的那条生命值还有架势，是有一定宽度的，读取了图像数据之后就是一个有多行数据的数组，但我们在计算的时候不需要那么多行，毕竟每行的数据其实都是差不多的，所以选取其中一行数据就可以了
    - 下面 get_value(target_img) 中的参数 **target_img** 就是这其中的一行数据，也是所谓的一维数组。
    - 在传入函数前， **target_img** 已经做过图像阈值处理，就是范围外的像素点处理成 0 了，所以计算从左数第一个出现的0的左边有多少个保留值，得到的结果就是我们需要的那个状态值
        - e.g. \[113, 113, 113, 115, 116, 110, 0, 0, 231, 0\] 计算结果为 6
        - e.g. \[0, 113, 123, 123, 105, 0, 0, 142, 115, 111\] 计算结果为 0
        - e.g. \[113, 0, 123, 123, 105, 0, 0, 142, 115, 111\] 计算结果为 1

### 1.3.2 架势的获取

（参考CSDN上的[OpenCV-Python教程（8、Canny边缘检测）](https://blog.csdn.net/sunny2038/article/details/9202641)）

- 架势和生命不同，它是从中间开始往两边增加的，但我们没有必要两边都获取，只需要一边就够了，这里我选择的是中间和右边。
- 还有几个问题比较麻烦，就是架势在归零的时候会不显示，积累到一定时候颜色会变化。
- 不过好在架势出现时中间有条白线，其像素点的值也集中在一定范围。所以要是这点的值进入这个范围，就判断为架势开始积累了
- 架势出现后，通过Canny边缘检测找到架势的边缘
- 量化

- 旧版
    1. ~~截取图像中架势的区域~~
    2. ~~由于架势的颜色也是偏向于红色（相比蓝色和绿色），所以进行颜色通道分离，取R(Red)通道的图像~~
    3. ~~通过观察发现，架势条中间那条白色的线所在的像素点的值大多集中在某范围内(例如自身架势的白线在架势积累较少时主要集中在 156 ~ 159)，还有 另外一个某范围内(例如自身架势的白线在架势积累较多时主要集中在 244 ~ 248)，将其视为架势条正在显示的标志，触发开启条件。~~
    4. ~~同样做图像阈值处理，处理完成后，获取数值~~
- 新版
    1. 截取图像中架势的区域
    2. 进行颜色通道分离，取R(Red)通道的图像
    3. while_line（白线，准确的说应该是白点。）出现后才开始边缘检测

~~~python
# 获取自身架势
def get_Self_Posture(img):
    img_roi = roi(img, x=401, x_w=490, y=389, y_h=389+1)    # x, x_w, y, y_h 获取自 get_vertices.py
    b, g ,r =cv2.split(img_roi)    # 颜色通道分离

    white_line = r[0][0]
    if 155 < white_line < 170 or white_line > 250:
        canny = cv2.Canny(cv2.GaussianBlur(r,(3,3),0), 0, 100)    # Canny边缘检测
        Self_Posture =  np.argmax(canny)
    else:
        Self_Posture = 0

    if white_line > 250 and Self_Posture < 10:    # 防止满架势却读取为 0
        Self_Posture == len(canny)

    return Self_Posture

# 获取目标架势
def get_Target_Posture(img):
    img_roi = roi(img, x=401, x_w=553, y=29, y_h=29+1)    # x, x_w, y, y_h 获取自 get_vertices.py
    b, g ,r =cv2.split(img_roi)    # 颜色通道分离

    white_line = r[0][0]
    if white_line > 190:
        canny = cv2.Canny(cv2.GaussianBlur(r,(3,3),0), 0, 100)    # Canny边缘检测
        Target_Posture =  np.argmax(canny)
    else:
        Target_Posture = 0

    if white_line > 250 and Target_Posture < 10:    # 防止满架势却读取为 0
        Target_Posture == len(canny)

    return Target_Posture
~~~

# 2 按键部分
- get_keys.py (捕获键盘的按键)
- direct_keys.py (控制键盘的按键)
- actions.py (动作控制)

## 2.1 捕获键盘的按键

查看代码 [get_keys.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/key_tools/get_keys.py)

**注：这部分是我从别人那里借鉴而来的**  
[源代码](https://github.com/Sentdex/pygta5/blob/master/getkeys.py)

检测按键的部分，用的是 virtual keys 。

~~~python
# virtual keys
vk = {
    'T' : 0x54,
    'P' : 0x50
}
~~~

vk 是从 keys.py 中截取的部分键值对，里面的按键我已经定义好了，如果你想定义更多的按键，可以在 [keys.py](https://github.com/Sentdex/pygta5/blob/master/keys.py) 中第 146 ~ 254行找到相应的按键然后添加进去

然后，调用 key_check() 时就会检测当前正在按的键
~~~python
def key_check():
    keys = []
    for key in ['T', 'P']:
        if wapi.GetAsyncKeyState(vk[key]):
            keys.append(key)
    return keys
~~~

## 2.2 控制键盘的按键

查看代码 [direct_keys.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/key_tools/direct_keys.py)

**注：这部分是我从别人那里借鉴而来的**  
[源代码](https://github.com/Sentdex/pygta5/blob/master/directkeys.py)

PressKey 压键  
ReleaseKey 松键  

具体用法看 动作控制（actions.py）

## 2.3 动作控制

查看代码 [actions.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/key_tools/actions.py)

**注：这部分是我从别人那里借鉴而来的**  
[源代码](https://github.com/Sentdex/pygta5/blob/master/3.%20test_model.py)

输出按键的部分，用的是 direct keys 

### 2.3.1 定义执行相应动作的函数

程序输出按键的部分，用的是 direct keys 。

~~~python
# direct keys
dk = {
    'W' : 0x11,
    'S' : 0x1F,
    'A' : 0x1E,
    'D' : 0x20,
    'LSHIFT' : 0x2A,
    'SPACE'  : 0x39,

    'Y' : 0x15,

    'J' : 0x24,
    'K' : 0x25,
}
~~~

dk 也是从 keys.py 中截取的部分键值对，里面的按键我已经定义好了，如果你想定义更多的按键，可以在 [keys.py](https://github.com/Sentdex/pygta5/blob/master/keys.py) 中第 35 ~ 143行找到相应的按键然后添加进去

动作定义
~~~python
# 先在 dk 里定义好相应的按键 
dk = {'W' : 0x11}

# 然后按照下面的示例定义执行相应动作的函数 
def Move_Forward():    # 移动 前
    PressKey(dk['W'])      # 压键
    time.sleep(0.1)    # 按键时长（控制长按短按）
    ReleaseKey(dk['W'])    # 松键
~~~

调用时，游戏就会接收到程序发送过来的按键信号，然后驱动操作对象做出相应的动作

### 2.3.2 act(values)

~~~python
def act(action=4, WS=2, AD=2):
    
    if   action == 0:
        act = Attack       # 攻击
    elif action == 1:
        act = Deflect      # 弹反
    elif action == 2:
        act = Step_Dodge   # 垫步
    elif action == 3:
        act = Jump         # 跳跃
    else:
        act = NOKEY        # 无键, 无动作
    
    act()
~~~